# Data & Training

In [2]:
def parse_dataset(length=256):
    directory = ""  #"latents\\" #"reconstruction_test_latents\\" #mtg-jamendo
    num_per = 1000
    count = 1

    for start in tqdm(range(0, 16150, num_per)):
        mtg_dataset, masks = retrieve_data("E:\SongsDataset\\mtg-jamendo\\", directory, start=start, count=num_per, sample_length=length)
        torch.save(mtg_dataset, f"E:\\SongsDataset\\length_{length}\\dataset{count}.pt")
        torch.save(masks, f"E:\\SongsDataset\\length_{length}\\dataset{count}-masks.pt")
        count += 1

    for start in tqdm(range(0, 3975, num_per)):
        spotify_dataset, masks  = retrieve_data("E:\\SongsDataset\\latents\\", directory, start=start, count=num_per, sample_length=length)
        torch.save(spotify_dataset, f"E:\\SongsDataset\\length_{length}\\dataset{count}.pt")
        torch.save(masks, f"E:\\SongsDataset\\length_{length}\\dataset{count}-masks.pt")
        count += 1

    full_dataset = torch.load(f"E:\\SongsDataset\\length_{length}\\dataset1.pt")
    full_masks = torch.load(f"E:\\SongsDataset\\length_{length}\\dataset1-masks.pt")

    for start in tqdm(range(2, 21)):
        new_data = torch.load(f"E:\\SongsDataset\\length_{length}\\dataset{start}.pt")
        new_masks = torch.load(f"E:\\SongsDataset\\length_{length}\\dataset{start}-masks.pt")

        full_dataset = torch.cat((full_dataset, new_data))
        full_masks = torch.cat((full_masks, new_masks))

    bool_masks = [[0 if (length - i) > x else 1 for i in range(length)] for x in full_masks]
    bool_masks = torch.tensor(bool_masks).bool()

    torch.save(full_dataset, f"E:\\SongsDataset\\length_{length}\\full_dataset.pt")
    torch.save(bool_masks, f"E:\\SongsDataset\\length_{length}\\full_masks.pt")

In [3]:
parse_dataset(256)

  0%|          | 0/17 [00:00<?, ?it/s]E:\Coding\SongAnalyzer\Analyzer\src\Data.py:63: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\pytorch\torch\csrc\utils\tensor_new.cpp:257.)
  tens = torch.tensor(dataset).to(torch.float32)
100%|██████████| 19/19 [04:51<00:00, 15.35s/it]


In [1]:
import torch.optim as optim
from torch.utils.data import DataLoader
import numpy as np
from tqdm import tqdm

import torch

from Training import train, evaluate
from Data import AudioDataset, retrieve_data

In [2]:
# ==== Model & Optimizer ====
num_heads = 16
num_layers = 16
d_model = 256
latent_space = 512
dim_feedforward = 1024
sample_length = 256
projection_dim = 128
dropout = 0.1
batch_size = 10

device = "cuda"

In [3]:
#dataset = torch.load(f"E:\\SongsDataset\\length_{sample_length}\\full_dataset.pt")
#masks = torch.load(f"E:\\SongsDataset\\length_{sample_length}\\full_masks.pt")

dataset = torch.load(f"E:\\SongsDataset\\length_{sample_length}\\dataset1.pt")
masks = torch.load(f"E:\\SongsDataset\\length_{sample_length}\\dataset1-masks.pt").bool()

In [4]:
from torch.utils.data import random_split, TensorDataset

num_samples, seq_length, embed_dim = dataset.shape

collective_dataset = AudioDataset(dataset, masks)

train_len = int(len(collective_dataset) * 0.9)
train_set, test_set = random_split(collective_dataset, [train_len, len(collective_dataset) - train_len])

train_dataloader = DataLoader(train_set, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(test_set, batch_size=batch_size, shuffle=True)

In [6]:
from Loss import combined_loss
from AudioTransformerLatentAttention import AudioTransformerLatentAttention

model = AudioTransformerLatentAttention(d_model=d_model, projection_dim=projection_dim, num_heads=num_heads, transformer_layers=num_layers, dim_feedforward=dim_feedforward, latent_space=latent_space, length=sample_length, dropout=dropout, name_extension="-fft-cos-256", use_rope=True, use_alibi=True)
model.set_checkpointing(False)

In [ ]:
from Loss import combined_loss
from AudioTransformer import AudioTransformer

model = AudioTransformer(d_model=d_model, num_heads=num_heads, transformer_layers=num_layers, dim_feedforward=dim_feedforward, latent_space=latent_space, length=sample_length, dropout=dropout, name_extension="-fft-cos-bidirectional", use_rope=True, use_alibi=True)
model.set_checkpointing(False)

In [7]:
model_parameters = filter(lambda p: p.requires_grad, model.parameters())
params = sum([np.prod(p.size()) for p in model_parameters])
print(params)

94150464


In [8]:
model = model.to(device)
optimizer = optim.AdamW(model.parameters(), lr=3e-5, weight_decay=0)

In [10]:
train(model, train_dataloader, test_dataloader, optimizer, num_epochs=10, device=device, loss_func=combined_loss)
evaluate(model, test_dataloader)

100%|██████████| 2059/2059 [02:16<00:00, 15.03it/s]


Epoch [1/10], Training Loss: 1.4206 	 Validation Loss: 1.3447


 28%|██▊       | 5099/18530 [17:18<45:34,  4.91it/s]  


KeyboardInterrupt: 

In [12]:
train(model, train_dataloader, test_dataloader, optimizer, num_epochs=10, device=device, loss_func=combined_loss)
evaluate(model, test_dataloader)

  0%|          | 0/1009 [00:00<?, ?it/s]

Source Shape: torch.Size([10, 256, 256])
q Shape: torch.Size([10, 256, 256])
q Shape: torch.Size([10, 16, 256, 16])
k Shape: torch.Size([10, 256, 256])
k Shape: torch.Size([10, 16, 256, 16])
v Shape: torch.Size([10, 256, 256])
v Shape: torch.Size([10, 16, 256, 16])


  0%|          | 0/1009 [00:00<?, ?it/s]


RuntimeError: masked_fill only supports boolean masks, but got dtype Float

In [12]:
train(model, train_dataloader, test_dataloader, optimizer, num_epochs=10, device=device, loss_func=combined_loss)
evaluate(model, test_dataloader)

  0%|          | 0/40 [00:00<?, ?it/s]E:\Coding\SongAnalyzer\.venv\lib\site-packages\torch\_dynamo\eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
  0%|          | 0/5 [00:00<?, ?it/s]E:\Coding\SongAnalyzer\.venv\lib\site-packages\torch\utils\checkpoint.py:87: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(
100%|██████████| 5/5 [00:02<00:00,  2.14it/s]


Epoch [1/10], Training Loss: 1.7508 	 Validation Loss: 1.5583


100%|██████████| 5/5 [00:02<00:00,  2.16it/s]


Epoch [2/10], Training Loss: 1.5815 	 Validation Loss: 1.4251


100%|██████████| 5/5 [00:02<00:00,  2.16it/s]


...

100%|██████████| 5/5 [00:02<00:00,  2.16it/s]


Epoch [10/10], Training Loss: 1.3108 	 Validation Loss: 1.3046


100%|██████████| 5/5 [00:02<00:00,  2.15it/s]


Avg Reconstructive Loss: 0.5965870023
Avg Cross FFT Loss: 0.7100642323
Avg Contrastive Loss: 1.3066511154


tensor(2.6133, device='cuda:0')

In [8]:
train(model, train_dataloader, test_dataloader, optimizer, num_epochs=10, device=device, loss_func=combined_loss)
evaluate(model, test_dataloader)

  0%|          | 0/40 [00:00<?, ?it/s]E:\Coding\SongAnalyzer\.venv\lib\site-packages\torch\_dynamo\eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
  0%|          | 0/5 [00:00<?, ?it/s]E:\Coding\SongAnalyzer\.venv\lib\site-packages\torch\utils\checkpoint.py:87: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(
100%|██████████| 5/5 [00:02<00:00,  2.36it/s]


Epoch [1/10], Training Loss: 1.7499 	 Validation Loss: 1.5711


100%|██████████| 5/5 [00:02<00:00,  2.34it/s]


Epoch [2/10], Training Loss: 1.5855 	 Validation Loss: 1.4284


 18%|█▊        | 7/40 [00:16<01:16,  2.33s/it]


KeyboardInterrupt: 

# Output Stuff

In [ ]:
from torch.utils.data import random_split

reconstruction_examples = retrieve_data("E:\SongsDataset\\",  "reconstruction_test_latents\\", sample_length=256)

In [ ]:
song_set, _ = random_split(reconstruction_examples, [len(reconstruction_examples), 0])
song_dataset = AudioDataset(song_set)
song_dataloader = DataLoader(song_dataset, batch_size=batch_size, shuffle=True)

In [2]:
model = torch.load(
    f"AudioTransformer-LatentSpace512-Heads16-TrasformerLayers16-DModel256-Dropout0.1-fft-cos-256\-Epoch-10.pt",
        weights_only=False)
evaluate(model, song_dataset)

NameError: name 'song_dataset' is not defined

In [ ]:
from Loss import combined_loss

device = "cuda"

model.eval()  # Set model to evaluation mode
model.to(device)
total_loss = 0.0
num_batches = 0

new_song = []

# Disable gradient computation for evaluation
with torch.no_grad():
    for batch in tqdm(song_dataloader):
        batch = batch.to(device)

        # Forward pass
        reconstructed = model(batch)

        new_song.extend(reconstructed.to("cpu"))

        loss = combined_loss(reconstructed, batch)

        total_loss += loss.item()
        num_batches += 1

l = np.array(np.stack(new_song)).reshape(64, -1)
np.save("reconstructed_song-2D-256-Campfire.npy", l)

In [ ]:
device = "cuda"

model.eval()
model.to(device)

total_loss = 0.0
num_batches = 0

latent_space = []

# Disable gradient computation for evaluation
with torch.no_grad():
    for batch in tqdm(song_dataloader):
        batch = batch.to(device)

        # Forward pass
        latent = model.to_latent(batch)

        latent_space.extend(latent.to("cpu"))

        num_batches += 1

In [ ]:
print(len(new_song))
print(len(new_song[0]))
print(len(new_song[0][0]))

In [ ]:
import IPython
from music2latent import EncoderDecoder
import numpy as np

encdec = EncoderDecoder()

In [ ]:
l = np.load("reconstructed_song-2D-256-Banana.npy")

In [ ]:
# compressed_song = np.load("reconstructed_song-256-FFT.npy")
wv_rec = encdec.decode(l)

In [ ]:
IPython.display.display(IPython.display.Audio(wv_rec, rate=44100))

# Data Output

In [1]:
from tqdm import tqdm
import torch
from Data import AudioDataset, retrieve_data

AudioTransformer(
  (projection): Linear(in_features=64, out_features=256, bias=True)
  (projection_gelu): GELU(approximate='none')
  (encoder): RoPEALiBiTransformerEncoder(
    (layers): ModuleList(
      (0-15): 16 x RoPEALiBiTransformerEncoderLayer(
        (self_attn): RoPEALiBiMultiheadAttention(
          (q_proj): Linear(in_features=256, out_features=256, bias=True)
          (k_proj): Linear(in_features=256, out_features=256, bias=True)
          (v_proj): Linear(in_features=256, out_features=256, bias=True)
          (out_proj): Linear(in_features=256, out_features=256, bias=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (linear1): Linear(in_features=256, out_features=1024, bias=True)
        (linear2): Linear(in_features=1024, out_features=256, bias=True)
        (norm1): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
        (norm2): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
        (dropout1): Dropout(p=0.1, inplace=False)
    

In [ ]:
directory = ""
spotify_dataset = retrieve_data("E:\\SongsDataset\\latents\\", directory, sample_length=256, keep_song_data_option=True)

In [31]:
def compress_song_average(song, model, mask=None):
    with torch.no_grad():
        latent_space = model.to_latent(song, mask)
        return torch.sum(latent_space.to('cpu'), dim=0) / len(song)

In [32]:
from Data import chunk_song
import os

def compute(model, name, length=256):
    model.to("cuda")
    model.eval()

    path = "E:\\SongsDataset\\latents\\"
    all_folders = os.listdir(path)

    file = open(f"output_analysis\\output-{name}.csv", "w", encoding='utf-8')

    for each_song in tqdm(all_folders):
        song_path = os.path.join(path, each_song)

        padded_data, zeros  = chunk_song(song_path, length)
        input_tensor = torch.Tensor(padded_data).reshape(-1, length, 64).to("cuda")

        mask = [0 for _ in range(padded_data.shape[0] // length - 1)]
        mask.append(zeros)
        bool_masks = [[0 if (length - i) > x else 1 for i in range(length)] for x in mask]
        bool_masks = torch.tensor(bool_masks).bool()
        bool_masks = bool_masks.to("cuda")

        latent = compress_song_average(input_tensor, model, mask=bool_masks)

        output = ""
        for value in latent:
            output += f"{str(value.item())} "

        file.write(output + f"\"{each_song}\"\n")

In [ ]:
model = torch.load("final-models\\AudioTransformerDeepCNN-LatentSpace512-Heads8-TrasformerLayers8-DModel256-Dropout0.1-fft\\-Epoch-10.pt", weights_only=False)
compute(model, "CNN-FINAL")

In [ ]:
model = torch.load("final-models\\AudioTransformerDeepCNN-LatentSpace512-Heads8-TrasformerLayers8-DModel256-Dropout0.1-fft\\-Epoch-10.pt", weights_only=False)
compute(model, "CNN-FINAL")

In [ ]:
model = torch.load("final-models\\AudioTransformerSingleLinearReconstruction-LatentSpace512-Heads16-TrasformerLayers16-DModel256-Dropout0.1-fft\\-Epoch-10.pt", weights_only=False)
compute(model, "Linear-FINAL")

In [8]:
model = torch.load("E:\Coding\SongAnalyzer\Analyzer\Webscraper\AudioTransformer-LatentSpace512-Heads8-TrasformerLayers8-DModel1024-Dropout0.1-fft-cos-\-Epoch-9.pt", weights_only=False)
model.set_checkpointing(False)
compute(model, "Linear-1024-TEST")

100%|██████████| 3973/3973 [02:25<00:00, 27.29it/s]


In [33]:
model = torch.load("AudioTransformer-LatentSpace512-Heads16-TrasformerLayers16-DModel256-Dropout0.1-fft-cos-256\-Epoch-10.pt", weights_only=False) # Set model to evaluation mode
compute(model, "Linear-256-Mask", length=256)

100%|██████████| 3973/3973 [03:00<00:00, 22.02it/s]


# Data Analysis